# Introduction

* Weighted graph
* Directed and Undirected graphs
* Acylic and cyclic graphs
* Sparse graph(more nodes than edges)
* Dense graph( more edges than nodes)

Graphs representations
* Adjacency matrix (`n x n` where n is the number of nodes).
Adjacency matrix is recommended for small densely connected graphs.

* Adjacency list. But checking if two nodes are connected requires a traversal
when using adjacency lists.

In [1]:
// scales better, memory efficient
// Note that we use immutable List here
// edges of each node is better represented using a linked list.
case class Node(val value: Int, val edges: List[Node])

// adjacency list. Adding new nodes becomes costly
// This is where we use tries to store all the nodes.
val graphsNodes = Array[Node]()  

defined class Node

Functional programming principles

* reduce side effects
* Referential transparent(idempotency) - Same result for the
same input.
* Immutable as much as possible. Immutable structures are thread safe.
* Functional way makes code concise and expressive. Less buggy and more readability.


In Scala, functions are first class citizens. Provides immutable
data structures. Its also supports object oriented programming. Interoperate
with Java.

* In graph's adjacency list representation, the list of nodes are commonly 
implemented as an array. But in functional programming, we wanted to
use immutable structures as much as possible. But with immutable
structures, the whole array might need to be copied and that would
affect the performance.

* Instead we could use something called as persistent structures like tries
to represent the graph's nodes. With tries, we can create immutable
structure with minimal copying(copy on the root)

In [13]:
trait Graph[V] {
    
    def vertices: List[V]
    
    def edges: List[(V, V)]
    
    // Graph needs to be immutable
    def addEdge(a: V, b: V): Graph[V]
    
    def neighbors(vertex: V): List[V]
}

defined trait Graph

In [14]:
// Directed Graph
// Maps in Scala are backed by a trie

// companion object
object Graph {
    
    def apply[V](adjList: Map[V, List[V]]): Graph[V] = new DirectedGraph(adjList)
    
    def apply[V](): Graph[V] = new DirectedGraph(Map[V, List[V]]())
}

class DirectedGraph[V](private val adjacencyList: Map[V, List[V]]) 
    extends Graph[V] {
    
    override def vertices: List[V] = adjacencyList.keys.toList    

    override def edges: List[(V, V)] = {
            adjacencyList.flatMap(entry => {
                val (v, neighbors) = entry
                neighbors.map(n => (v, n))
            }).toList
    }
    
    // Graph needs to be immutable
    override def addEdge(a: V, b: V): Graph[V] = {
        val neighbors =  b :: adjacencyList.getOrElse(a,
                                                     List.empty[V])
        new DirectedGraph(adjacencyList + (a -> neighbors))
    }
    
    override def neighbors(vertex: V): List[V] =  {
        adjacencyList.getOrElse(vertex, Nil)
    }
}

defined object Graph
defined class DirectedGraph

In [8]:
class UndirectedGraph[V](adjacencyList: Map[V, List[V]])
    extends DirectedGraph[V](adjacencyList) { 

    override def addEdge(a: V, b: V): Graph[V] = {
        // This is shorter but seems little bit inefficient
        // we create two graphs one for adding each edge
        // super.addEdge(a, b).addEdge(b, a)
        
        val aNeighbors = b :: adjacencyList.getOrElse(a, Nil)
        val bNeighbors = a :: adjacencyList.getOrElse(b, Nil)
        
        new UndirectedGraph(adjacencyList + (
        (a -> aNeighbors),
        (b -> bNeighbors)
        ))
    }
}

defined class UndirectedGraph

In [12]:
case class WeightedEdge[V](destination: V, weight: Int)

class WeightedGraph[V](
    private val adjacencyList:Map[V, List[WeightedEdge[V]]]
) extends Graph[V] {

    override def vertices: List[V] = adjacencyList.keys.toList    

    override def edges: List[(V, V)] = {
            adjacencyList.flatMap(entry => {
                val (v, neighbors) = entry
                neighbors.map(n => (v, n.destination))
            }).toList
    }
    
    def addEdge(a: V, weightedEdge: WeightedEdge[V]): Graph[V] = {
        val neighbors =  weightedEdge :: adjacencyList.getOrElse(a, Nil)
        new WeightedGraph(adjacencyList + (a -> neighbors))

    }
    
    // Graph needs to be immutable
    override def addEdge(a: V, b: V): Graph[V] = {
        addEdge(a, WeightedEdge(b, 0))
    }
    
    override def neighbors(vertex: V): List[V] =  {
        adjacencyList.getOrElse(vertex, Nil).map(_.destination)
    }

    def neighborWeights(vertex: V): List[WeightedEdge[V]] =  {
        adjacencyList.getOrElse(vertex, Nil).toList
    }

}

defined class WeightedEdge
defined class WeightedGraph

## Depth first search

* Traversing a graph is going through every node in the graph

In [15]:
val sampleGraph = new DirectedGraph[String](
    Map(("a", List("b", "g")),
        ("b", List("c")),
        ("c", List("d", "e")),
        ("d", List("e")),
        ("e", List()),
        ("f", List("a")),
        ("g", List("h")),
        ("h", List("f"))
        )
)

sampleGraph: DirectedGraph[String] = ammonite.$sess.cmd13$Helper$DirectedGraph@7b562132

In [20]:
// depth first traversal
// But this is not tail recursive

def traverse(graph: Graph[String], 
             start: String, 
             visitedNodes: List[String]) {

    val currentVisitedNodes = visitedNodes ++ List(start)

    val unvisitedNeighbors = graph.neighbors(start)
    .filter(!currentVisitedNodes.contains(_))

    if (unvisitedNeighbors.length > 0)
        unvisitedNeighbors.foreach(n => {
            traverse(graph, n, currentVisitedNodes)
        })
    else
        println(currentVisitedNodes.mkString("->"))
}

def dfs(graph: Graph[String], start: String): Unit = {    
    traverse(graph, start, List[String]())
}

dfs(sampleGraph, "a")

a->b->c->d->e
a->b->c->e
a->g->h->f


defined function traverse
defined function dfs

In [23]:
// v2
// to traverse deep graphs, we need to either use tail recursion
// or use an iterative version

import scala.collection.mutable.{Stack, ListBuffer}

def dfsIterative(graph: Graph[String], start: String): Unit = {
    val stack = Stack[(String, List[String])]()
    stack.push(start -> List[String]())
    
    while (!stack.isEmpty) {
        val (c, path) = stack.pop()
        val cpath = path ++ List(c)
        val neighbors = graph.neighbors(c)
        val unvisitedNeighbors = neighbors.filter(!path.contains(_))
        if (unvisitedNeighbors.length > 0) {
            unvisitedNeighbors.foreach(n => 
                                       stack.push((n -> cpath)))
        } else {
            println(cpath.mkString("->"))
        }
    }   
}


dfsIterative(sampleGraph, "a")

a->g->h->f
a->b->c->e
a->b->c->d->e


import scala.collection.mutable.{Stack, ListBuffer}


defined function dfsIterative